# Basic PINN demo
This notebook shows how to train a simple Physics-Informed Neural Network using `kfac_pinn`.

In [ ]:
import jax.numpy as jnp
from jax import random
import equinox as eqx
from kfac_pinn import KFAC, pinn

key = random.PRNGKey(0)
model = pinn.make_mlp(key=key)
opt = KFAC(lr=1e-3)
state = opt.init(model, jnp.ones((10, 1)))
